In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [15]:
# Read the data
time_data = pd.read_csv('C:/Users/user/Desktop/R24-066/Component 04/Backend/Dataset/time_series_data_02.csv')

In [16]:
# Convert 'Date' column to datetime
time_data['Date'] = pd.to_datetime(time_data['Date'])

In [17]:
# Define custom thresholds for high and low defect rates (specific point or level used as a reference for making decisions)
high_threshold = 0.05  # Modify this threshold as needed
low_threshold = 0.01   # Modify this threshold as needed

In [18]:
# Initialize variables to store highest and lowest defect rates
highest_defect = {'Weekday': None, 'Defect_Type': None, 'Defect_Rate': -float('inf')}
lowest_defect = {'Weekday': None, 'Defect_Type': None, 'Defect_Rate': float('inf')}

In [19]:
# List of defect types and weekdays
defect_types = ['Openseam', 'Runoff', 'SPI Errors', 'High Low']
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [28]:
# Loop through each defect type
for i, defect_type in enumerate(defect_types):
    # Initialize lists to store defect rates and weekdays
    actual_defect_rates = []
    forecasted_defect_rates = []
    defect_weekdays = []

    # Loop through each weekday
    for weekday in weekdays:
        # Filter data for the current defect type and weekday
        data_subset = time_data[(time_data['Date'].dt.day_name() == weekday) & (time_data['Defect_Type'] == defect_type)]

        # Split data into features and target variable
        X = data_subset[['Date']]
        y = data_subset['Defect_Rate']

        # Initialize and train the model
        model = RandomForestRegressor()
        model.fit(X, y)

        # Forecast for the next weekday
        future_date = X['Date'].max() + pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri')
        future_prediction = model.predict(pd.DataFrame({'Date': [future_date]}))[0]

        # Update highest and lowest defect rates
        if future_prediction > highest_defect['Defect_Rate']:
            highest_defect['Weekday'] = weekday
            highest_defect['Defect_Type'] = defect_type
            highest_defect['Defect_Rate'] = future_prediction
        if future_prediction < lowest_defect['Defect_Rate']:
            lowest_defect['Weekday'] = weekday
            lowest_defect['Defect_Type'] = defect_type
            lowest_defect['Defect_Rate'] = future_prediction

        # Append defect rate and weekday to lists
        if not data_subset.empty:
            actual_defect_rates.append(data_subset['Defect_Rate'].values[0])
        else:
            actual_defect_rates.append(None)
        forecasted_defect_rates.append(future_prediction)
        defect_weekdays.append(weekday)

# Print the results with only three decimal places and including the defect type
print("Highest Defect Rate: {:.3f} for {}".format(highest_defect['Defect_Rate'], highest_defect['Defect_Type']))
print("Lowest Defect Rate: {:.3f} for {}".format(lowest_defect['Defect_Rate'], lowest_defect['Defect_Type']))

Highest Defect Rate: 0.787 for Runoff
Lowest Defect Rate: 0.071 for Openseam
